In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer
import os

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# 데이터 불러오기
train_data = pd.read_excel('detailed_split_data.xlsx')  # 파일 경로 수정
train_data.head()

,Q,q,A
0,사이트 정보,비드큐 사이트,www.bidq.co.kr
1,사이트 정보,비드큐 고객센터,1544-6714
2,사이트 정보,비드큐 팩스 번호,0505-490-8800
3,사이트 결제,정보 이용료의 경우,"1개월 30,000원, 3개월 85,000원, 6개월 148,000원, 12개월 2..."
4,사이트 결제,낙찰 연구소의 경우,"정보 이용료가 포함되어 있으며, 1개월 60,000원, 3개월 170,000원, 6..."


In [4]:
# 사전 학습된 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 데이터 전처리: Q, A, B 쌍으로 분리
qa_pairs = []
for idx, row in train_data.iterrows():
    for i in range(0, len(row), 3):  # 3칸씩 이동하면서 Q, A, B를 읽음
        if pd.notna(row[i]) and pd.notna(row[i+1]) and pd.notna(row[i+2]):
            qa_pairs.append((row[i], row[i+1], row[i+2]))

# 새로운 데이터프레임 생성
df = pd.DataFrame(qa_pairs, columns=['Q', 'A', 'B'])

# 질문 임베딩 및 저장
df['embedding'] = df.apply(lambda row: model.encode(row.Q), axis=1)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# 코사인 유사도 계산 함수
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

In [6]:
# 답변 반환 함수
def return_answers(question, df, model):
    embedding = model.encode(question)
    df['score'] = df.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    best_match = df.loc[df['score'].idxmax()]

    # 매칭된 질문에 해당하는 답변 필터링
    relevant_answers = df[df['Q'] == best_match['Q']]

    answers = []
    for idx, row in relevant_answers.iterrows():
        answers.append({
            'answer': row['A'],  # 답변
            'detail': row['B']   # 상세 내용
        })

    return best_match['Q'], answers

In [7]:
# 함수 테스트
print(return_answers('2단계경쟁', df, model))

('입찰 용어', [{'answer': '일반경쟁', 'detail': '입찰참가자격에 제한을 두지 않는 입찰로서 법령 등에 의하여 허가ᆞ인가ᆞ면허ᆞ등록ᆞ신고 등을 받았거나 당해 자격요건에 적합해야합니다.'}, {'answer': '제한경쟁', 'detail': '입찰시 지역, 실적, 시공능력, 유자격자명부 등으로 입찰참가자격에 제한을 두는 입찰입니다.\n예) 지역제한입찰 : 당해 공사현장의 특별시ᆞ광역시ᆞ도에 본사를 둔 지역 업체만 입찰에 참여할 수 있는 제도입니다.'}, {'answer': '지명경쟁', 'detail': '특수기술, 전문적인 기술의 보유자가 아니면 공사 진행이 원활치 않을 경우 참가자격에 제한을 두는 제도, 추정가격 3억 이하 종합공사, 추정가격 1억이하 그 밖의 공사, 추정가격 1억이하 물품제조'}, {'answer': '2단계경쟁', 'detail': '1단계경쟁 입찰을 통과한 몇몇의 업체를 대상으로 2단계경쟁 입찰에 부치는 것을 말합니다. 주로 입찰참가자격 사전심사(PQ)입찰이 이에 속하는데 시공실적 및 신기술 보유 등 엄격한 입찰참가 자격을 통과한 업체를 대상으로 1단계 경쟁을 통과한 업체들만 별도 2단계 설계평가, 기술능력평가, 경영상태 평가, 입찰가격 평가 등 2단계경쟁 입찰에 부치는 것을 말합니다.'}, {'answer': '총액입찰', 'detail': '입찰서에 입찰금액만을 기재하여 입찰하는 제도이며 낙찰된 회사는 착공계 제출시 산출내역서를 함께 제출하여야 합니다.'}, {'answer': '내역입찰', 'detail': '총액입찰과 반대로 입찰서에 입찰금액과 발주기관에서 배부한 공종별 물량내역서에 단가를 기재한 산출내역서를 함께 제출하는 제도입니다.(100억 이상 입찰시 적용)'}, {'answer': '수의계약', 'detail': '주로 관내로 공고가 나오며 적격심사 평가 없이 1순위가 되면 바로 계약이 진행됩니다.'}, {'answer': '전자시담', 'detail': '정해진 업체들만 참가 가능하며 G2B상에서 

In [8]:
# 임베딩 저장
embeddings = np.vstack(df['embedding'].values)
np.save('embeddings.npy', embeddings)

# 원본 데이터와 임베딩 저장
df.to_pickle('train_data_with_embeddings.pkl')

In [9]:
# 임베딩 파일을 zip으로 압축
!zip -r /content/embeddings.npy.zip /content/embeddings.npy

  adding: content/embeddings.npy (deflated 68%)


In [10]:
!pip install streamlit transformers pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.1 MB/s eta 0:00:00


In [11]:
# Streamlit 애플리케이션
%%writefile app.py
import streamlit as st
import pandas as pd
from sentence_transformers import SentenceTransformer
from numpy import dot
from numpy.linalg import norm

# 데이터 불러오기
df = pd.read_pickle('train_data_with_embeddings.pkl')  # 파일 경로 수정

# 사전 학습된 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 코사인 유사도 계산 함수
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

# 답변 반환 함수
def return_answers(question, df):
    embedding = model.encode(question)
    df['score'] = df['embedding'].apply(lambda x: cos_sim(x, embedding))
    df_sorted = df.sort_values(by='score', ascending=False)

    matched_question = df_sorted.iloc[0]['Q']  # 가장 높은 유사도를 가진 질문
    answers = []

    for idx, row in df_sorted.iterrows():
        if row['Q'] == matched_question:
            answers.append({
                'answer': row['A'],  # 답변
                'detail': row['B']   # 상세 내용
            })

    return matched_question, answers

# Streamlit 웹 애플리케이션 제목
st.title("Infose")

# Streamlit 앱 초기 상태 설정
if 'matched_question' not in st.session_state:
    st.session_state.matched_question = ""
if 'answers' not in st.session_state:
    st.session_state.answers = []
if 'selected_answer' not in st.session_state:
    st.session_state.selected_answer = ""

# 사용자 입력 받기
user_input = st.text_input("질문을 입력하세요:")

if st.button("분석하기"):
    if user_input:
        # 데이터셋에서 답변 찾기
        matched_question, answers = return_answers(user_input, df)
        st.session_state.matched_question = matched_question
        st.session_state.answers = answers
        st.session_state.selected_answer = ""

        if answers:
            st.write(f"매칭된 질문: {matched_question}")
            options = [answer['answer'] for answer in answers]


# 사용자가 선택지를 변경할 때 자동으로 상세 내용을 표시
selected_answer = st.selectbox(
    "원하는 답변을 선택하세요:",
    options=[answer['answer'] for answer in st.session_state.answers],
    index=0 if st.session_state.selected_answer == "" else [answer['answer'] for answer in st.session_state.answers].index(st.session_state.selected_answer)
)

if selected_answer and selected_answer != st.session_state.selected_answer:
    st.write(f"선택한 답변: {selected_answer}")
    for answer in st.session_state.answers:
        if answer['answer'] == selected_answer:
            st.write(f"상세 내용: {answer['detail']}")
            break


Writing app.py


In [12]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.90.72.73


In [13]:
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 9.007s
your url is: https://dull-rockets-add.loca.lt
^C
